In [1]:
!apt update

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (141 kB/s)
Reading package lists... Done
Building dependency tree       
Reading s

In [2]:
!apt-get install python-opengl -y

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-opengl is already the newest version (3.1.0+dfsg-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 76 not upgraded.


In [3]:
!apt-get install -y xvfb python-opengl > /dev/null 2>&1


In [4]:
!pip install piglet

In [5]:
!pip install gym pyvirtualdisplay > /dev/null 2>&1


In [6]:
import gym
import numpy as np
import matplotlib.pyplot as plt
from IPython import display as ipythondisplay

In [7]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(400, 300))
display.start()

In [8]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [9]:
import os

os.chdir('./gdrive/My Drive/codes/Thesis/')

In [10]:
    import argparse
    import torch
    import os

    from thesis.planner.cem import CEM
    from thesis.main import load_checkpoint
    from thesis.main_helpers_init import init_env_from_args
    from thesis.util.logging import Log
    from thesis.environment.util import postprocess_observation


    log_path = './logs/planning_trajectory'
    log_args = argparse.Namespace()
    log_args.log_directory = log_path
    log_args.print_log = True

    log = Log(log_args)
    log.create_image_folder('trajectory')
    log.create_log('rewards', 'tau', 'r_true', 'r_pred')

    path = './logs/log_colab/iter_90_checkpoint.pth'
    checkpoint = load_checkpoint(path)

    args = checkpoint['args']
    rssm = checkpoint['env_model']

    init_state = checkpoint['init_state']

    env = init_env_from_args(args, ctx=locals())
    env.reset()
    env.set_state(init_state)
    rssm.reset()

    planner_args = argparse.Namespace()
    planner_args.planning_horizon = 12
    planner_args.num_plan_candidates = 100
    planner_args.num_plan_top_candidates = 10
    planner_args.num_plan_iter = 10
    planner_args.plan_batch_size = 1
    planner = CEM(planner_args)

    _, info = planner.plan(rssm, show_progress=True)

    trajectory = info['trajectory']

    actions = torch.chunk(trajectory, planner_args.planning_horizon, dim=1)

    for i, a in enumerate(actions):
        a = a.squeeze(1)
        o_true, r_true, _, _ = env.step(a)
        o_pred, r_pred, _, _ = rssm.step(a)

        o_true = postprocess_observation(o_true, args.bit_depth, dtype=torch.float32)[0]
        o_pred = postprocess_observation(o_pred, args.bit_depth, dtype=torch.float32)[0]

        r_true = r_true[0].item()
        r_pred = r_pred[0].item()

        log.log_values('rewards', i + 1, r_true, r_pred)
        log.log_observations('trajectory',
                             f'step_{i + 1}',
                             o_true,
                             o_pred,
                             )
        









Planner: 100%|██████████| 10/10 [00:08<00:00,  1.17it/s, iters done 10/10]
